# 1- import libraries 

In [1]:
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 2-  Decision Tree Class

In [2]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.tree = None
    
    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)
    
    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape

        # Stopping criteria
        if depth >= self.max_depth or num_samples < self.min_samples_split:
            leaf_value = self._calculate_leaf_value(y)
            return leaf_value

        # Select the best split
        feat_idx, threshold = self._best_split(X, y, num_features)
        if feat_idx is None:
            return self._calculate_leaf_value(y)
        
        # Split the data
        left_idxs, right_idxs = self._split(X[:, feat_idx], threshold)
        left_tree = self._build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right_tree = self._build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        
        return {"feature_index": feat_idx, "threshold": threshold, "left": left_tree, "right": right_tree}
    
    def _best_split(self, X, y, num_features):
        best_loss = np.inf
        split_idx, split_threshold = None, None
        
        for feat_idx in range(num_features):
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                loss = self._calculate_loss(X_column, y, threshold)
                if loss < best_loss:
                    best_loss = loss
                    split_idx = feat_idx
                    split_threshold = threshold
        
        return split_idx, split_threshold

    def _calculate_loss(self, X_column, y, threshold):
        # Split data
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return np.inf
        
        # Calculate the loss (mean squared error)
        left_loss = self._mean_squared_error(y[left_idxs])
        right_loss = self._mean_squared_error(y[right_idxs])
        
        num_left, num_right = len(left_idxs), len(right_idxs)
        total_loss = (num_left / len(y)) * left_loss + (num_right / len(y)) * right_loss
        
        return total_loss

    def _split(self, X_column, split_threshold):
        left_idxs = np.where(X_column <= split_threshold)[0]
        right_idxs = np.where(X_column > split_threshold)[0]
        return left_idxs, right_idxs

    def _mean_squared_error(self, y):
        mean_y = np.mean(y)
        mse = np.mean((y - mean_y) ** 2)
        return mse
    
    def _calculate_leaf_value(self, y):
        return np.mean(y)
    
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])

    def _traverse_tree(self, x, tree):
        if not isinstance(tree, dict):
            return tree
        
        feature_value = x[tree["feature_index"]]
        if feature_value <= tree["threshold"]:
            return self._traverse_tree(x, tree["left"])
        else:
            return self._traverse_tree(x, tree["right"])

# 3-  Random Forest Class

In [3]:
class RandomForestRegressor:
    def __init__(self, n_trees=10, max_depth=None, min_samples_split=2):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.trees = []
    
    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split)
            
            # Bootstrap sampling
            X_sample, y_sample = self._bootstrap_sample(X, y)
            
            # Fit the decision tree
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)
    
    def _bootstrap_sample(self, X, y):
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, size=n_samples, replace=True)
        return X[idxs], y[idxs]
    
    def predict(self, X):
        # Make predictions from all trees
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        
        # Average predictions across trees for regression
        return np.mean(tree_preds, axis=0)

# 4- implementation on California Housing dataset

In [4]:
if __name__ == "__main__":
    # Load the dataset
    data = fetch_california_housing()
    X, y = data.data, data.target

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and train the Random Forest Regressor
    forest = RandomForestRegressor(n_trees=10, max_depth=10)
    forest.fit(X_train, y_train)

    # Make predictions
    y_pred = forest.predict(X_test)

    # Evaluate the model's performance
    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error: {mse:.4f}")

Mean Squared Error: 0.3225
